# 농업 환경 변화에 따른 작물 병해 진단 AI 경진대회

## Import modules

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import json
from tqdm import tqdm
import pickle
import shutil

import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset,DataLoader
from torchvision.datasets.folder import default_loader
from torchvision import transforms as T
import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from utils import CosineAnnealingWarmUpRestarts, EarlyStopping

## Focus augmentation

바운딩 박스 좌표를 활용해 주변 배경을 제거한 객체 사진을 따로 저장하여 학습과정에서 데이터 증강기법으로 사용


In [2]:
# # 전체 복사
# for path in tqdm(glob.glob('data/train/*'),position=0):
#     shutil.copytree(path,f'{path}_focus')
# #각 파일 포맷들 이름 변경
# for sample in tqdm(glob.glob('data/train/*focus/*.csv'),position=0):
#     os.rename(sample,f'{sample[:-4]}_focus.csv')

# for sample in tqdm(glob.glob('data/train/*focus/*.json'),position=0):
#     os.rename(sample,f'{sample[:-5]}_focus.json')

# for sample in tqdm(glob.glob('data/train/*focus/*.jpg'),position=0):
#     os.rename(sample,f'{sample[:-4]}_focus.jpg')

# # 객체 영역만 저장
# for sample in tqdm(glob.glob('data/train/*focus'),position=0):
#     img_path = glob.glob(f'{sample}/*.jpg')[0]
#     sample_image = cv2.imread(img_path)
#     sample_json = json.load(open(glob.glob(f'{sample}/*.json')[0],'r'))
#     points = sample_json['annotations']['bbox'][0]
#     x= int(points['x'])
#     y= int(points['y'])
#     w= int(points['w'])
#     h= int(points['h'])
#     crop_focus = sample_image[y:y+h,x:x+w,:].copy()
#     cv2.imwrite(img_path,crop_focus)

## Csv feature

환경 변수에서 학습에 사용할 feature들을 선택 후 미리 최대값, 최솟값을 계산해 저장

In [3]:
# 활용할 환경변수 선택
# csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
#                 '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# csv_files = sorted(glob.glob('data/train/*/*.csv'))


# # feature 별 최대값, 최솟값 계산
# for csv in tqdm(csv_files[1:],position=0):
#     temp_csv = pd.read_csv(csv)[csv_features]
#     temp_csv = temp_csv.replace('-',np.nan).dropna()
#     if len(temp_csv) == 0:
#         continue
#     temp_csv = temp_csv.astype(float)
#     temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
#     max_arr = np.max([max_arr,temp_max], axis=0)
#     min_arr = np.min([min_arr,temp_min], axis=0)

# # feature 별 최대값, 최솟값 dictionary 생성
# csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}

# 구해 놓은 값 저장
# with open('csv_minmax.pickle','wb') as f:
#     pickle.dump(csv_feature_dict, f)

# 미리 저장한 값 읽어오기
with open('csv_minmax.pickle', 'rb') as f:
    csv_feature_dict = pickle.load(f)


CSV를 사용하여 label을 생성하기 위한 정보

In [4]:
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

In [5]:
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'


In [6]:
label_encoder = {key: idx for idx,key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}

## Dataset, DataLoader

In [7]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode ='train', csv_feature_dict=None,label_encoder=None,transform=None):
        self.mode = mode
        self.files = files
        self.csv_feature_dict = csv_feature_dict
        
        self.csv_feature_check = [0]*len(self.files)
        self.csv_features = [None]*len(self.files)
        
        self.max_len = 24 * 6
        self.label_encoder = label_encoder
        self.transform = transform
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('\\')[-1]
        
        #csv
        if self.csv_feature_check[i] == 0:
            csv_path = f'{file}/{file_name}.csv'
            df = pd.read_csv(csv_path)[self.csv_feature_dict.keys()]
            df = df.replace('-', 0)
            # MinMax scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
                df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])
            # zero padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]
            # transpose to sequential data
            csv_feature = pad.T
            self.csv_features[i] = csv_feature
            self.csv_feature_check[i] = 1
        else:
            csv_feature = self.csv_features[i]
        #image
        image_path = f'{file}/{file_name}.jpg'
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image_transform = self.transform(image=image)
            image = image_transform['image']
        
        if self.mode == 'train':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = f'{crop}_{disease}_{risk}'
            
            return {
                'img' : image,
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32),
                'label' : torch.tensor(self.label_encoder[label], dtype=torch.long)
            }
        elif self.mode =='val':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = f'{crop}_{disease}_{risk}'
            
            return {
                'img' : image,
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32),
                'label' : torch.tensor(self.label_encoder[label], dtype=torch.long)
            }
            
        else:
            return {
                'img' :image,
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32)
            }
          

In [8]:
train_origin = [path for path in sorted(glob.glob('data/train/*')) if not path.endswith('focus')]
train_focus = [path for path in sorted(glob.glob('data/train/*')) if path.endswith('focus')]

test = sorted(glob.glob('data/test/*'))

train_label = pd.read_csv('data/train.csv')['label']

train, val = train_test_split(train_origin, test_size=0.2, stratify=train_label)

train = train+train_focus

In [9]:
device = torch.device("cuda:0")
batch_size = 16
class_n = len(label_encoder)
learning_rate = 1e-4#0.025
embedding_dim = 512
num_features = len(csv_feature_dict)
max_len = 24*6
dropout_rate = 0.4
epochs = 1500
vision_pretrain = True
save_path = 'model.pt'

In [10]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

h,w = 512,384

train_transform = A.Compose([
    A.Resize(h, w), 
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Normalize(IMAGENET_MEAN,IMAGENET_STD),
    ToTensorV2()
])
test_transform = A.Compose([
    A.Resize(h, w), 
    A.Normalize(IMAGENET_MEAN,IMAGENET_STD),
    ToTensorV2()
])


train_dataset = CustomDataset(files=train,
                              mode='train',
                              csv_feature_dict=csv_feature_dict,
                              label_encoder=label_encoder,
                              transform=train_transform)

val_dataset = CustomDataset(files=val,
                            mode='val',
                            csv_feature_dict=csv_feature_dict,
                            label_encoder=label_encoder,
                            transform=test_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,  shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,  shuffle=False)

## Model
- pretrained model : densenet201 from torchvision.models

In [11]:
class CNN_Encoder(nn.Module):
    def __init__(self, class_n, rate=0.1):
        super(CNN_Encoder, self).__init__()
        self.model = models.densenet201(pretrained=True)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

In [12]:
class RNN_Decoder(nn.Module):
    def __init__(self, max_len, embedding_dim, num_features, class_n, rate):
        super(RNN_Decoder, self).__init__()
        self.lstm = nn.LSTM(max_len, embedding_dim)
        self.rnn_fc = nn.Linear(num_features*embedding_dim, 1000)
        self.pre_final_layer = nn.Linear(1000 + 1000, 512) # resnet out_dim + lstm out_dim
        self.final_layer = nn.Linear(512, class_n) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(rate)

    def forward(self, enc_out, dec_inp):
        hidden, _ = self.lstm(dec_inp)
        hidden = hidden.view(hidden.size(0), -1)
        hidden = self.rnn_fc(hidden)
        concat = torch.cat([enc_out, hidden], dim=1) # enc_out + hidden 
        fc_input = concat
        output = self.relu(self.dropout((self.pre_final_layer(fc_input))))
        output = self.final_layer(output)
        return output

In [13]:
class CNN2RNN(nn.Module):
    def __init__(self, max_len, embedding_dim, num_features, class_n, rate):
        super(CNN2RNN, self).__init__()
        self.cnn = CNN_Encoder(embedding_dim, rate)
        self.rnn = RNN_Decoder(max_len, embedding_dim, num_features, class_n, rate)
        
    def forward(self, img, seq):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq)
        
        return output

In [14]:
model = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model = model.to(device)

In [15]:
def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

In [16]:
def train_step(batch_item, training):
    img = batch_item['img'].to(device)
    csv_feature = batch_item['csv_feature'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img, csv_feature)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        score = accuracy_function(label, output)
        return loss, score
    else:
        model.eval()
        with torch.no_grad():
            output = model(img, csv_feature)
            loss = criterion(output, label)
        score = accuracy_function(label, output)
        return loss, score

# Model Training

In [ ]:
loss_plot, val_loss_plot = [], []
metric_plot, val_metric_plot = [], []
max_f1_score = 0
patience=50
early_stopping = EarlyStopping(patience = patience, verbose = True, path =save_path )

for epoch in range(epochs):
    lr = optimizer.param_groups[0]['lr']
    print(f'{epoch+1} epoch start LR : {lr:.2e}')
    total_loss, total_val_loss = 0, 0
    total_acc, total_val_acc = 0, 0
    
    with tqdm(enumerate(train_dataloader),total = len(train_dataloader), unit="batch",position=0) as tepoch:
        training = True
        for batch, batch_item in tepoch:
            batch_loss, batch_acc = train_step(batch_item, training)
            total_loss += batch_loss
            total_acc += batch_acc

            tepoch.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
                'Mean F-1' : '{:06f}'.format(total_acc/(batch+1))
            })
        loss_plot.append(total_loss/(batch+1))
        metric_plot.append(total_acc/(batch+1))
    
    with tqdm(enumerate(val_dataloader),total = len(val_dataloader), unit="batch",position=0) as tepoch:
        training = False
        for batch, batch_item in tepoch:
            batch_loss, batch_acc = train_step(batch_item, training)
            total_val_loss += batch_loss
            total_val_acc += batch_acc

            tepoch.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
                'Mean Val F-1' : '{:06f}'.format(total_val_acc/(batch+1))
            })
        val_loss_plot.append(total_val_loss/(batch+1))
        val_metric_plot.append(total_val_acc/(batch+1))
    
    
    scheduler.step(val_loss_plot[-1])
    
    
    early_stopping(-val_metric_plot[-1], model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    

# Model Evaluation

In [27]:
test_dataset = CustomDataset(files=test,
                            mode = 'test',
                            csv_feature_dict=csv_feature_dict,
                            label_encoder=label_encoder,
                            transform=test_transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=256,  shuffle=False)


In [28]:
save_path = 'model384.pt'

In [29]:
def predict(data_loader):
    model.eval()
    with tqdm(enumerate(data_loader),total = len(data_loader), unit="batch",position=0) as tepoch:
        results = []
        for batch, batch_item in tepoch:
            img = batch_item['img'].to(device)
            seq = batch_item['csv_feature'].to(device)
            with torch.no_grad():
                output = model(img, seq)
            output = torch.argmax(output,dim=1).detach().cpu().numpy()
            results.extend(output)
    return results



model = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model.load_state_dict(torch.load(save_path, map_location=device))
model = model.to(device)

preds = predict(test_dataloader)

100%|█████████████████████████████████████████████████████████████████████████████| 203/203 [18:51<00:00,  5.57s/batch]


In [34]:
preds_label = np.array([label_decoder[int(val)] for val in preds])
submission = pd.read_csv('data/sample_submission.csv')
submission['label'] = preds
submission.to_csv('data/submission.csv',index=False)